In [16]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime
import tensorflow as tf;
from six.moves import cPickle as pickle;
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
from pandas import read_csv, DataFrame
import math
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import Callback
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from pandas import Series
%matplotlib inline
py.init_notebook_mode(connected=True)

In [2]:
def get_quandl_data(quandl_id):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, authtoken="rGwyAH1yyw29yX8E1LQJ", returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df
    
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

In [3]:
exchanges = ['KRAKEN', 'COINBASE', 'BITFINEX']

exchange_data = {}

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    btc_exchange_df = btc_exchange_df.replace(0, np.NaN) # Wack Kraken values ˜
    exchange_data[exchange] = btc_exchange_df

print(exchange_data)

btc_usd_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')
print(list(exchange_data['COINBASE'].columns))

# Merging data together to create one consistent set 
# Basically does a left join on all three data sets and gets the mean of all values
# Prevents weird discrepencies in the data 

new = pd.merge(exchange_data['KRAKEN'], exchange_data['COINBASE'], how='outer', left_index=True, right_index=True)
new = pd.merge(new, exchange_data['BITFINEX'], how='outer', left_index=True, right_index=True)
new['new_open'] = new[['Open', 'Open_x', 'Open_y']].mean(axis=1)
new['new_high'] = new[['High_x', 'High_y', 'High']].mean(axis=1)
new['new_low'] = new[['Low_x', 'Low_y', 'Low']].mean(axis=1)
new['new_close'] = new[['Close_x', 'Close_y', 'Close']].mean(axis=1)
new['new_btc_volume'] = new[['Volume (BTC)_x', 'Volume (BTC)_y', 'Volume (BTC)']].mean(axis=1)
new['new_currency_volume'] = new[['Volume (Currency)_x', 'Volume (Currency)_y', 'Volume (Currency)']].mean(axis=1)
new['new_weighted_price'] = new[['Weighted Price_x', 'Weighted Price_y', 'Weighted Price']].mean(axis=1)

df = new[['new_open', 'new_high', 'new_low', 'new_close', 'new_btc_volume', 'new_currency_volume', 'new_weighted_price']]

print(df.head())

Loaded BCHARTS/KRAKENUSD from cache
Loaded BCHARTS/COINBASEUSD from cache
Loaded BCHARTS/BITFINEXUSD from cache
{'KRAKEN':                   Open        High         Low       Close  Volume (BTC)  \
Date                                                                       
2014-01-07   874.67040   892.06753   810.00000   810.00000     15.622378   
2014-01-08   810.00000   899.84281   788.00000   824.98287     19.182756   
2014-01-09   825.56345   870.00000   807.42084   841.86934      8.158335   
2014-01-10   839.99000   857.34056   817.00000   857.33056      8.024510   
2014-01-11   858.20000   918.05471   857.16554   899.84105     18.748285   
2014-01-12   899.96114   900.93989   833.00001   860.00000     25.429433   
2014-01-13   847.32152   859.99999   815.00000   835.00000     25.869127   
2014-01-14   835.00000   877.29300   805.00000   831.00000     31.662881   
2014-01-15   831.00000   864.00000   828.00000   850.00364      6.707565   
2014-01-16   853.00000   865.00000   824.

In [4]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('time_%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('time_%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('time_%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
            
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def create_rnn_set(data, look_back=1):
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in range(1, look_back + 1)]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    print(len(dataY))
    return np.array(dataX), np.array(dataY)

# Create the values and feed them into the model
def create_data(df, look_back=1):
    values = df['new_weighted_price'].values.reshape(-1,1)
    values = values.astype('float32')
    scaler = MinMaxScaler(feature_range=(0, 1))
    values = scaler.fit_transform(values)

    train_size = int(len(values) * 0.7)
    test_size = len(values) - train_size
    train, test = values[0:train_size,:], values[train_size:len(values),:]

    val = series_to_supervised(train, look_back).values
    trainX, trainY = val[:,0:look_back], val[:, look_back]
    trainX = trainX.reshape(trainX.shape[0], look_back, 1)

    val = series_to_supervised(test, look_back).values
    testX, testY = val[:,0:look_back], val[:, look_back]
    testX = testX.reshape(testX.shape[0], look_back, 1)
    
    return trainX, trainY, testX, testY, scaler

In [5]:

# def prepare_sequences(x_train, y_train, window_length):
#     windows = []
#     windows_y = []
#     for i, sequence in enumerate(x_train):
#         len_seq = len(sequence)
#         for window_start in range(0, len_seq - window_length + 1):
#             window_end = window_start + window_length
#             window = sequence[window_start:window_end]
#             windows.append(window)
#             windows_y.append(y_train[i])
#     return np.array(windows), np.array(windows_y)

# USE_SEQUENCES = False
# USE_STATELESS_MODEL = False

# # you can all the four possible combinations
# # USE_SEQUENCES and USE_STATELESS_MODEL

# max_len = 20
# batch_size = 1

# N_train = 1000
# N_test = 200

# X_train = np.zeros((N_train, max_len))
# X_test = np.zeros((N_test, max_len))

# print('X_train shape:', X_train.shape)
# print('X_test shape:', X_test.shape)

# y_train = np.zeros((N_train, 1))
# y_test = np.zeros((N_test, 1))

# one_indexes = choice(a=N_train, size=int(N_train / 2, replace=False)
# X_train[one_indexes, 0] = 1
# y_train[one_indexes] = 1

# one_indexes = choice(a=N_test, size=N_test / 2, replace=False)
# X_test[one_indexes, 0] = 1
# y_test[one_indexes] = 1


# class ResetStatesCallback(Callback):
#     def __init__(self):
#         self.counter = 0

#     def on_batch_begin(self, batch, logs={}):
#         if self.counter % max_len == 0:
#             self.model.reset_states()
#         self.counter += 1


# if USE_SEQUENCES:
#     max_len = 10
#     X_train, y_train = prepare_sequences(X_train, y_train, window_length=max_len)
#     X_test, y_test = prepare_sequences(X_test, y_test, window_length=max_len)

# X_train = np.expand_dims(X_train, axis=2)  # input dim is 1. Timesteps is the sequence length.
# X_test = np.expand_dims(X_test, axis=2)

# print('sequences_x_train shape:', X_train.shape)
# print('sequences_y_train shape:', y_train.shape)

# print('sequences_x_test shape:', X_test.shape)
# print('sequences_y_test shape:', y_test.shape)

# if USE_STATELESS_MODEL:
#     print('Build STATELESS model...')
#     model = Sequential()
#     model.add(LSTM(10, input_shape=(max_len, 1), return_sequences=False))
#     model.add(Dense(1, activation='sigmoid'))

#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#     print('Train...')
#     model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
#               validation_data=(X_test, y_test), shuffle=False, callbacks=[ResetStatesCallback()])

#     score, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
#     print('___________________________________')
#     print('Test score:', score)
#     print('Test accuracy:', acc)
# else:
#     # STATEFUL MODEL
#     print('Build STATEFUL model...')
#     model = Sequential()
#     model.add(LSTM(10,
#                    batch_input_shape=(1, 1, 1), return_sequences=False,
#                    stateful=True))
#     model.add(Dense(1, activation='sigmoid'))

#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#     x = np.expand_dims(np.expand_dims(X_train.flatten(), axis=1), axis=1)
#     y = np.expand_dims(np.array([[v] * max_len for v in y_train.flatten()]).flatten(), axis=1)
#     model.fit(x,
#               y,
#               callbacks=[ResetStatesCallback()],
#               batch_size=1,
#               shuffle=False)

#     print('Train...')
#     for epoch in range(15):
#         mean_tr_acc = []
#         mean_tr_loss = []
#         for i in range(len(X_train)):
#             y_true = y_train[i]
#             for j in range(max_len):
#                 tr_loss, tr_acc = model.train_on_batch(np.expand_dims(np.expand_dims(X_train[i][j], axis=1), axis=1),
#                                                        np.array([y_true]))
#                 mean_tr_acc.append(tr_acc)
#                 mean_tr_loss.append(tr_loss)
#             model.reset_states()

#         print('accuracy training = {}'.format(np.mean(mean_tr_acc)))
#         print('loss training = {}'.format(np.mean(mean_tr_loss)))
#         print('___________________________________')

#         mean_te_acc = []
#         mean_te_loss = []
#         for i in range(len(X_test)):
#             for j in range(max_len):
#                 te_loss, te_acc = model.test_on_batch(np.expand_dims(np.expand_dims(X_test[i][j], axis=1), axis=1),
#                                                       y_test[i])
#                 mean_te_acc.append(te_acc)
#                 mean_te_loss.append(te_loss)
#             model.reset_states()

#             for j in range(max_len):
#                 y_pred = model.predict_on_batch(np.expand_dims(np.expand_dims(X_test[i][j], axis=1), axis=1))
#             model.reset_states()

#         print('accuracy testing = {}'.format(np.mean(mean_te_acc)))
#         print('loss testing = {}'.format(np.mean(mean_te_loss)))
#         print('___________________________________')

In [6]:
max_len = 3
batch_size = 1

trainX, trainY, testX, testY, scaler = create_data(df, look_back=3)

trainX = trainX[26:,:,:]
trainY = trainY[26:]
testX = testX[3:,:,:]
testY = testY[3:]

class ResetStatesCallback(Callback):
    def __init__(self):
        self.counter = 0

    def on_batch_begin(self, batch, logs={}):
        if self.counter % max_len == 0:
            self.model.reset_states()
        self.counter += 1


# trainX = np.expand_dims(trainX, axis=2)  # input dim is 1. Timesteps is the sequence length.
# testX = np.expand_dims(testX, axis=2)

print('sequences_trainX shape:', trainX.shape)
print('sequences_trainY shape:', trainY.shape)

print('sequences_testX shape:', testX.shape)
print('sequences_testY shape:', testY.shape)


print('Build STATEFUL model...')
model = Sequential()
model.add(LSTM(100,
               batch_input_shape=(1, 1, 1), stateful=True))
model.add(Dense(1))

model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])

x = np.expand_dims(np.expand_dims(trainX.flatten(), axis=1), axis=1)
y = np.expand_dims(np.array([[v] * max_len for v in trainY.flatten()]).flatten(), axis=1)

print('Train...')
for epoch in range(15):
    mean_tr_acc = []
    mean_tr_loss = []
    for i in range(len(trainX)):
        y_true = trainY[i]
        for j in range(max_len):
            tr_loss, tr_acc = model.train_on_batch(np.expand_dims(np.expand_dims(trainX[i][j], axis=1), axis=1),
                                                   np.array([y_true]))
            mean_tr_acc.append(tr_acc)
            mean_tr_loss.append(tr_loss)
        model.reset_states()

    print('accuracy training = {}'.format(np.mean(mean_tr_acc)))
    print('loss training = {}'.format(np.mean(mean_tr_loss)))
    print('___________________________________')

    mean_te_acc = []
    mean_te_loss = []
    for i in range(len(testX)):
        for j in range(max_len):
            te_loss, te_acc = model.test_on_batch(np.expand_dims(np.expand_dims(testX[i][j], axis=1), axis=1),
                                                  testY[i].reshape(1,1))
            mean_te_acc.append(te_acc)
            mean_te_loss.append(te_loss)
        model.reset_states()

        for j in range(max_len):
            y_pred = model.predict_on_batch(np.expand_dims(np.expand_dims(testX[i][j], axis=1), axis=1))
        model.reset_states()

    print('accuracy testing = {}'.format(np.mean(mean_te_acc)))
    print('loss testing = {}'.format(np.mean(mean_te_loss)))
    print('___________________________________')

In [45]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

data=df['new_weighted_price'].values
dif = difference(data, 1)
predict_chart = go.Scatter(x=np.arange(0, len(dif)), y=dif, name= 'Scaled_data')
py.iplot([predict_chart])

In [115]:
# Based on: https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/

# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

# Return to the original values 
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        print('Epoch: ', i)
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
    return model

def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]

def create_predictions(lstm_model, test_s, raw_values):
    predictions = list()
    for i in range(len(test_s)):
        # make one-step forecast
        X, y = test_s[i, 0:-1], test_s[i, -1]
        yhat = forecast_lstm(lstm_model, 1, X)
        # invert scaling
        yhat = invert_scale(scaler, X, yhat)
        # invert differencing
        yhat = inverse_difference(raw_values, yhat, len(test_s)+1-i)
        # store forecast
        predictions.append(yhat)
        expected = raw_values[len(train) + i + 1]
#         print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))
        
    return predictions


raw_values =df['new_weighted_price'].values
dif = difference(raw_values, 1).values.reshape(-1,1)

values = create_rnn_set(dif, 1).values

train_size = int(len(values) * 0.7)
test_size = len(values) - train_size
train, test = values[0:train_size,:], values[train_size:len(values),:]

scaler, train_s, test_s = scale(train, test)
lstm_model = fit_lstm(train_s, batch_size=1, nb_epoch=50, neurons=50)

# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_s[:, 0].reshape(len(train_s), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)

predictions = create_predictions(lstm_model, test_s, raw_values)
    
# report performance
rmse = math.sqrt(mean_squared_error(raw_values[train_size+1:len(raw_values)], predictions))
print('Test RMSE: %.3f' % rmse)
expected = raw_values[train_size:len(values)]

predict_chart = go.Scatter(x=np.arange(0, len(predictions)), y=predictions, name= 'Predicted Data')
expected_chart = go.Scatter(x=np.arange(0, len(expected)), y=expected, name= 'Expected Data')

py.iplot([predict_chart, expected_chart])

Epoch:  0
Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Epoch:  12
Epoch:  13
Epoch:  14
Epoch:  15
Epoch:  16
Epoch:  17
Epoch:  18
Epoch:  19
Epoch:  20
Epoch:  21
Epoch:  22
Epoch:  23
Epoch:  24
Epoch:  25
Epoch:  26
Epoch:  27
Epoch:  28
Epoch:  29
Epoch:  30
Epoch:  31
Epoch:  32
Epoch:  33
Epoch:  34
Epoch:  35
Epoch:  36
Epoch:  37
Epoch:  38
Epoch:  39
Epoch:  40
Epoch:  41
Epoch:  42
Epoch:  43
Epoch:  44
Epoch:  45
Epoch:  46
Epoch:  47
Epoch:  48
Epoch:  49
Test RMSE: 88.693


In [114]:
# Create a multi-step predictor
def create_data(df, n_lag, n_seq):
    raw_values = df['new_weighted_price'].values
    dif = difference(raw_values, 3).values
    dif = dif.reshape(len(dif), 1)
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_values = scaler.fit_transform(dif)
    scaled_values = scaled_values.reshape(len(scaled_values), 1)
    
    supervised = series_to_supervised(scaled_values, n_lag, n_seq)
    supervised_values = supervised.values
    
    train_size = int(len(supervised_values) * 0.7)
    test_size = len(supervised_values) - train_size
    train, test = supervised_values[0:train_size,:], supervised_values[train_size:len(supervised_values),:]
    
    return scaler, train, test

def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
    # reshape training into [samples, timesteps, features]
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    # design network
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    # fit network
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
        model.reset_states()
    return model

def forecast_lstm(model, X, n_batch):
    # reshape input pattern to [samples, timesteps, features]
    X = X.reshape(1, 1, len(X))
    # make forecast
    forecast = model.predict(X, batch_size=n_batch)
    # convert to array
    return [x for x in forecast[0, :]]

def inverse_difference(last_ob, forecast):
    # invert first forecast
    inverted = list()
    inverted.append(forecast[0] + last_ob)
    # propagate difference forecast using inverted first value
    for i in range(1, len(forecast)):
        inverted.append(forecast[i] + inverted[i-1])
    return inverted

def make_forecasts(model, n_batch, train, test, n_lag, n_seq):
    forecasts = list()
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
        # make forecast
        forecast = forecast_lstm(model, X, n_batch)
        # store the forecast
        forecasts.append(forecast)
    return forecasts

# evaluate the RMSE for each forecast time step
def evaluate_forecasts(test, forecasts, n_lag, n_seq):
    for i in range(n_seq):
        actual = [row[i] for row in test]
        predicted = [forecast[i] for forecast in forecasts]
        rmse = math.sqrt(mean_squared_error(actual, predicted))
        print('t+%d RMSE: %f' % ((i+1), rmse))
 
# plot the forecasts in the context of the original dataset
def plot_forecasts(series, forecasts, n_test):
    expected = go.Scatter(x=np.arange(0, len(series)), y=series, name='Expected')
    plots = [expected]
    for i in range(0,len(forecasts), 100):
        off_s = len(series) - n_test + i - 1
        off_e = off_s + len(forecasts[i]) + 1
        xaxis = [x for x in range(off_s, off_e)]
        yaxis = [series[off_s]] + forecasts[i]
        chart = go.Scatter(x=xaxis, y=yaxis, name='point')
        plots.append(chart)

    py.iplot(plots)
    
# inverse data transform on forecasts
def inverse_transform(series, forecasts, scaler, n_test):
    inverted = list()
    for i in range(len(forecasts)):
        # create array from forecast
        forecast = np.array(forecasts[i])
        forecast = forecast.reshape(1, len(forecast))
        # invert scaling
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0, :]
        # invert differencing
        index = len(series) - n_test + i - 1
        last_ob = series[index]
        inv_diff = inverse_difference(last_ob, inv_scale)
        # store
        inverted.append(inv_diff)
    return inverted


n_lag = 1
n_seq = 3
n_batch = 1
n_epochs = 50
n_neurons = 50

raw_data = df['new_weighted_price'].values
train_size = int(len(raw_data) * 0.7)
test_size = len(raw_data) - train_size

scaler, train, test = create_data(df, n_lag, n_seq)
model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons)

forecast = make_forecasts(model, n_batch, train, test, n_lag, n_seq)
forecasts = inverse_transform(raw_data, forecast, scaler, test_size+2)
actual = [row[n_lag:] for row in test]
actual = inverse_transform(raw_data, actual, scaler, test_size+2)
# evaluate forecasts
evaluate_forecasts(actual, forecasts, n_lag, n_seq)
# plot forecasts
plot_forecasts(raw_data, forecasts, test_size+2)
    

t+1 RMSE: 151.768294
t+2 RMSE: 301.646647
t+3 RMSE: 440.369311
